In [10]:
import numpy as np
import os
import time

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
train_ds, test_ds = tfds.load('cifar10', split=['train','test'], as_supervised=True)

In [12]:
def normalize_resize(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.divide(image, 255)
    image = tf.image.resize(image, (28, 28))
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    return image, label 

train = train_ds.map(normalize_resize).cache().map(augment).shuffle(100).batch(64).repeat()
test = test_ds.map(normalize_resize).cache().batch(64)

In [13]:
model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28, 3)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    train,
    epochs=5,
    steps_per_epoch=60000 // 64,
    validation_data=test, verbose=2
)

Epoch 1/5
937/937 - 8s - loss: 1.9736 - accuracy: 0.2751 - val_loss: 1.7908 - val_accuracy: 0.3662 - 8s/epoch - 8ms/step
Epoch 2/5
937/937 - 6s - loss: 1.8707 - accuracy: 0.3179 - val_loss: 1.7608 - val_accuracy: 0.3669 - 6s/epoch - 6ms/step
Epoch 3/5
937/937 - 5s - loss: 1.8387 - accuracy: 0.3290 - val_loss: 1.7139 - val_accuracy: 0.3981 - 5s/epoch - 6ms/step
Epoch 4/5
937/937 - 5s - loss: 1.8177 - accuracy: 0.3391 - val_loss: 1.7144 - val_accuracy: 0.3906 - 5s/epoch - 6ms/step
Epoch 5/5
937/937 - 5s - loss: 1.7993 - accuracy: 0.3492 - val_loss: 1.6795 - val_accuracy: 0.4089 - 5s/epoch - 6ms/step
